In [27]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

In [28]:
# Loading the dataset

filename = "311_Service_Requests_2yrs.csv"
df = pd.read_csv(filename)
display(df.head(5))

,service_request_id,requested_date,updated_date,closed_date,status_description,source,service_name,agency_responsible,address,comm_code,comm_name,location_type,longitude,latitude,point
0,23-00000797,2023/01/02 12:00:00 AM,2023/01/10 12:00:00 AM,2023/01/10 12:00:00 AM,Closed,Other,Finance - ONLINE TIPP Agreement Request,CFOD - Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23-00001045,2023/01/02 12:00:00 AM,2024/01/11 12:00:00 AM,2024/01/11 12:00:00 AM,Closed,Other,Active Living Program Application,CS - Recreation and Social Programs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23-00001163,2023/01/02 12:00:00 AM,2023/01/06 12:00:00 AM,2023/01/06 12:00:00 AM,Closed,Phone,CN - Registered Social Worker Letter,CS - Calgary Neighbourhoods,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23-00001191,2023/01/02 12:00:00 AM,2024/05/19 12:00:00 AM,2023/01/10 12:00:00 AM,Closed,Other,CT - Lost Property,OS - Calgary Transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23-00001584,2023/01/02 12:00:00 AM,2023/01/04 12:00:00 AM,2023/01/04 12:00:00 AM,Closed,Other,Recreation - Arena Booking Application,CS - Calgary Recreation,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Cleaning: Drop rows

df = df.iloc[:1062842]

In [30]:
#Shape

df.shape

(1062842, 15)

In [31]:
#Columns

df.columns

Index(['service_request_id', 'requested_date', 'updated_date', 'closed_date',
       'status_description', 'source', 'service_name', 'agency_responsible',
       'address', 'comm_code', 'comm_name', 'location_type', 'longitude',
       'latitude', 'point'],
      dtype='object')

In [32]:
#Datatypes

df.dtypes

service_request_id     object
requested_date         object
updated_date           object
closed_date            object
status_description     object
source                 object
service_name           object
agency_responsible     object
address               float64
comm_code              object
comm_name              object
location_type          object
longitude             float64
latitude              float64
point                  object
dtype: object

In [33]:
# Identifying number of missing values 

missing_values = df.isna().sum()
missing_values

service_request_id          0
requested_date              0
updated_date                0
closed_date             32093
status_description          0
source                      0
service_name                0
agency_responsible        158
address               1062842
comm_code               73834
comm_name               73833
location_type           73626
longitude               73845
latitude                73845
point                   73845
dtype: int64

In [34]:
# Dropping address column

df = df.drop('address',axis = 1)
df.shape

(1062842, 14)

In [35]:
# Deriving new columns from requested date

df['requested_date'] = pd.to_datetime(df['requested_date'], format = '%Y/%m/%d %I:%M:%S %p')
print(f"Data type of 'requested_date': {df['requested_date'].dtype}")

df['request_year'] = df['requested_date'].dt.year
df['request_month'] = df['requested_date'].dt.month
df['request_day'] = df['requested_date'].dt.day


Data type of 'requested_date': datetime64[ns]


In [36]:
df['updated_date'] = pd.to_datetime(df['updated_date'], format = '%Y/%m/%d %I:%M:%S %p')

df['update_year'] = df['updated_date'].dt.year
df['update_month'] = df['updated_date'].dt.month
df['update_day'] = df['updated_date'].dt.day


In [37]:
# Deriving new columns from requested date

df['closed_date'] = pd.to_datetime(df['closed_date'], format = '%Y/%m/%d %I:%M:%S %p')
print("Dataype:", df['closed_date'].dtype)

# Converting null values to NaT
df['closed_date'] = df['closed_date'].fillna(pd.NaT)


df['closed_year'] = df['closed_date'].dt.year
df['closed_month'] = df['closed_date'].dt.month
df['closed_day'] = df['closed_date'].dt.day

# Replacing null values in derived columns with 0 and converting the column values to int type

df.loc[df['closed_date'].isna(), ['closed_year', 'closed_month', 'closed_day']] = 0
df[['closed_year', 'closed_month', 'closed_day']] = df[['closed_year', 'closed_month', 'closed_day']].astype('Int32')

#display(df.head(5))

Dataype: datetime64[ns]


In [38]:
# Checking datatypes ofconverted columns

df.dtypes

service_request_id            object
requested_date        datetime64[ns]
updated_date          datetime64[ns]
closed_date           datetime64[ns]
status_description            object
source                        object
service_name                  object
agency_responsible            object
comm_code                     object
comm_name                     object
location_type                 object
longitude                    float64
latitude                     float64
point                         object
request_year                   int32
request_month                  int32
request_day                    int32
update_year                    int32
update_month                   int32
update_day                     int32
closed_year                    Int32
closed_month                   Int32
closed_day                     Int32
dtype: object

In [39]:
# Handling Missing Data
columnNameDropped = missingDataPercentage[missingDataPercentage >= 40].index.tolist()
print("\nColumns with missing percentage more than 40% missing values are:", columnNameDropped)
df = df.drop(columns = missingDataPercentage[missingDataPercentage > 40].index)

# Handling Unwanted Data
beforeCount = df.shape[0]
df = df[(df['requested_date'] < '2025-01-01') & (df['requested_date'] > '2023-01-01')]
afterCount =df.shape[0]
deletedCount = beforeCount - afterCount
print(f"\nCount of deleted request which are recieved on or after 2025-01-01 and before 2023-01-01: {deletedCount}")

#Handling Missing Community Code
communityNames = df[df['comm_code'].isnull() & df['comm_name'].notnull()]['comm_name']
print(f"\nCommunity name with community code null and community name exists: {communityNames}")

df['comm_code'].fillna(df['comm_name'], inplace=True)
print(f"\nCommunity Code is filled with Community name for {communityNames} community")

#Handling Missing Longitude and Latitude with their median 
df['longitude'] = df['longitude'].fillna(df['longitude'].median())
df['latitude'] = df['latitude'].fillna(df['latitude'].median())
print("\nLongitude and latitude missing values are replaced with its corresponding median")

#Handling Missing Point with the mode
df['point'] = df['point'].fillna(df['point'].mode()[0])
print("\nPoint missing values are replaced with its mode")

NameError: name 'missingDataPercentage' is not defined

In [ ]:
# Calculating closing delay and creating new inttype column for closing delay

df['response_time'] = df['closed_date'] - df['requested_date']
print("1",df['response_time'].dtype)
df['response_time'] = df['response_time'].dt.days
print(df['response_time'].dtype)
df['response_time'] = df['response_time'].astype('Int64')

print(df['response_time'].dtype)

#Check
#df_subset = df.iloc[1050:1116]  # Python slicing includes 150 but excludes 166
#display(df_subset)

In [ ]:
# Identifying duplicate requests using regex and creating new column 

df['duplicate_request'] = df['status_description'].str.contains(r'Duplicate \(Closed\)', regex=True)

# Convert the boolean values to 'Yes'/'No'
df['duplicate_request'] = df['duplicate_request'].replace({True: 'Yes', False: 'No'})

# Check
#df_subset = df.iloc[150:166]  # Python slicing includes 150 but excludes 166
#display(df_subset)


In [ ]:
# Check max date value in requested_date column

max_value = df['requested_date'].max()
print(max_value)

In [ ]:
# Season Categorisation of "Requests"

# Defining Calgary's timezone
calgary_tz = pytz.timezone('America/Edmonton')  

# Exact UTC times for solstices and equinoxes (taken from Govt of Canada Website)

seasons_utc = {
    'Spring_2023': '2023-03-20 21:24:00',
    'Summer_2023': '2023-06-21 14:57:00',
    'Autumn_2023': '2023-09-23 06:50:00',
    'Winter_2023': '2023-12-22 03:27:00',
    'Spring_2024': '2024-03-20 03:06:00',
    'Summer_2024': '2024-06-20 20:50:00',
    'Autumn_2024': '2024-09-22 12:43:00',
    'Winter_2024': '2024-12-21 09:20:00'
}

# Converting the UTC times to Calgary local time

seasons = {}

for season, utc_time_str in seasons_utc.items():
    
    # Converting the UTC string into a datetime object
    
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
    utc_time = pytz.utc.localize(utc_time) 
    
    # Converting to Calgary local time
    local_time = utc_time.astimezone(calgary_tz)
    
    # Saving the result in the dictionary
    seasons[season] = local_time
    
for key, value in seasons.items():
#print(f"{key}: {value.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{key}: {value}")

In [ ]:

# Keeping the local time but making it aware for requested_date columns

if df['requested_date'].dt.tz is None:
    df['new_requested_date'] = df['requested_date'].dt.tz_localize('America/Edmonton')

print(df['new_requested_date'].head())

In [ ]:
# Categorizing into seasons and creating a new 'season' column

# Assigning seasons based on request date

def get_season(request_date):
    for season, season_date in seasons.items():
        if request_date < season_date:
            return season
    return 'Winter_2024'  # Default to the latest season

# Creating new season column 

df['Season'] = df['new_requested_date'].apply(get_season)

display(df)

In [ ]:
# Closed requests taken into account

statistics_response_time = df['response_time'].describe()
print(statistics_response_time)

In [ ]:
# Checking no of empty values in 'response_time' column

na_count = df['response_time'].isna().sum()
print(na_count)

In [ ]:
# Checking number of open requests 

no_open_requests = len(df[df['status_description'] == 'Open'])
no_open_requests

In [ ]:
# Finding data discrepencies (688 numbers because the missing values in closed_dates was found to be 32093)

filtered_df = df[df['status_description'] == 'Open' ]
grouped_data = filtered_df.groupby('status_description')['closed_date']
pd.set_option('display.max_rows', None)
#display(grouped_data.tail(200))

In [ ]:
# To solve this data discrepency, change the "status" of requests with closed dates to "Closed".

df['modified_status'] = df.apply(
   lambda row: 'Closed' if pd.notna(row['closed_date']) and row['status_description'] == 'Open' 
    else ('Duplicate (Closed)' if pd.notna(row['closed_date']) and row['status_description'] == 'Duplicate (Open)' 
          else row['status_description']), axis=1
)
display(df.tail(20))

In [ ]:
summary = df['status_description'].value_counts()
print(summary)

In [ ]:
summary1 = df['modified_status'].value_counts()
print(summary1)

In [ ]:
#Add column for Community Sector using the community sector csv file
community_data=pd.read_csv("/Users/jincythomas/Desktop/PROJECT-DATA601/311ServiceRequests/CSV_SECTORS.csv")
def merge_community_sector(main_data, community_data):
    # Rename the relevant columns in the community_data for clarity and consistency
    community_data.rename(columns={'COMM_CODE': 'comm_code', 'SECTOR': 'community_sector'}, inplace=True)

    # Merge the datasets based on the 'comm_code'
    merged_data = main_data.merge(community_data[['comm_code', 'community_sector']], on='comm_code', how='left')

    return merged_data

df = merge_community_sector(df, community_data)
print("\n\033[1m"+"Additional Columns created are:"+"\033[0m")
print("\tcommunity_sector")

#Handling Missing for Community related columns
df['comm_code'].fillna("Community Centrepoint", inplace=True)
df['comm_name'].fillna("Community Centrepoint", inplace=True)
df['community_sector'].fillna("Community Centrepoint", inplace=True)

In [ ]:
# Add a column for Divisions of Agency assigned for the requests

#Unassigned agencies are assigned to corresponding divisions
df.loc[df['agency_responsible'].isnull() & df['service_name'].str.contains('WATR -'), 'agency_responsible'] = 'UEP - Utilities & Environmental Protection'
df.loc[df['agency_responsible'].isnull() & df['service_name'].str.contains('PSD -'), 'agency_responsible'] = 'PDS - Planning & Development Services'
df.loc[df['agency_responsible'].isnull() & df['service_name'].str.contains('CPI -'), 'agency_responsible'] = 'OSC - Operational Services and Compliance'

# agency abbreviations are extracted
def extract_division(value):
    if pd.isna(value):
        return np.nan
    parts = value.split('-')
    resultStr = parts[0].strip() if '-' in value else value.strip()
    return resultStr


df['agency_division'] = df['agency_responsible'].apply(extract_division)
#Actual agencies or divisions under Calgary Government
agency_division = {
    'agency_name': ['Affiliated Organizations', 'Chief Financial Officer Department', 'Corporate Wide Service Requests',
                    'Calgary Police & Fire Services', 'Community Services', "Deputy City Manager's Office",
                   'Elected Officials', 'Fleet and Inventory', 'Information Services','Legal or Legislative Services',
                   'Office of the City Auditor','Operational Services and Compliance', 'Partnerships',
                   'Planning & Development Services','Project Information and Control Systems', 'Recreation and Social Programs',
                    'Transportation', 'Utilities & Environmental Protection'],
    'abbreviations': [['AO', 'Affiliated Organizations'], ['CFOD'], ['Corporate Wide Service Requests'], 
                      ['CPFS'],['CS'], ['DCMO'], 
                      ['Elected Officials'], ['Fleet and Inventory'], ['IS'], ['LL','LLSS'],
                      ['Office of the City Auditor'],['OS','OSC'],['Partnerships'],
                      ['PD','PDS'],['PICS'],['Recreation and Social Programs'],
                      ['TRAN','Tranc'], ['UEP','Uepc']]
}


# Create a mapping dictionary
mapping = {abbreviation: agency_name 
           for agency_name, abbreviations in zip(agency_division['agency_name'], agency_division['abbreviations']) 
           for abbreviation in abbreviations}


# Replace the agency_division values with actual agency_name or divisions
df['agency_division'] = df['agency_division'].map(mapping)

#noDivisionDF = df[df['agency_division'].isnull()]
#display(noDivisionDF)

agencies= df['agency_division'].unique()
    
# Iterate through each agency division in the list
for division in agencies:
    subset_df = df[df['agency_division'] == division]
    
    # Split the 'agency_responsible' column at the first hyphen and create 'agency_subdivision'
    df.loc[df['agency_division'] == division, 'agency_subdivision'] = subset_df['agency_responsible'].apply(
        lambda x: x.split('-', 1)[1] if '-' in x else division
    )

    # Split the 'service_name' column at the first hyphen and create 'service_category'
    df.loc[df['agency_division'] == division, 'service_category'] = subset_df['service_name'].apply(
        lambda x: x.split('-', 1)[0] if '-' in x else x
    )

    # Split the 'service_name' column at the first hyphen and create 'service_request'
    df.loc[df['agency_division'] == division, 'service_request'] = subset_df['service_name'].apply(
        lambda x: x.split('-', 1)[1] if '-' in x else x
    )
    
# Display the updated DataFrame
#print("Updated DataFrame:")
display(df.head(100))


print("\n\033[1m"+"Additional Columns created are:"+"\033[0m")
print("\tagency_division")
print("\tagency_subdivision")
print("\tservice_category")
print("\tservice_request")

## Response Efficiency

To answer this we have to consider the requests which are not duplicate, and which has closed date>= requested_date

### Questions?

• Which agency handles the most and least number of service requests?

• What is the average response rate and time for resolving for service requests for each agency?

• Who are the most efficient agencies in terms of response and resolution times?

• How does the response efficiency vary across different years?

• How does the response efficiency vary across different years for each agency divisions?

In [40]:
#Filter the records from your dataframe df where closed_date is greater than or equal to requested_date, 
#closed_date is not null, and duplicate_request is 'No'

print("\n\033[1m"+"Agency division and the count of requests handles by each division:"+"\033[0m")
efficiencyDF = df[(df['closed_date'] >= df['requested_date']) &
                 (df['closed_date'].notna()) & 
                 (df['duplicate_request'] == 'No')]
print(f"For answering the response efficiency, we have considered {efficiencyDF.shape[0]} requests")

#The most and least request handled agencies
grouped_counts = efficiencyDF.groupby(['agency_division']).size().reset_index(name='count')
sorted_counts = grouped_counts.sort_values(by='count', ascending=False)
first_row = sorted_counts.head(1).to_string(index=False, header=False)
last_row = sorted_counts.tail(1).to_string(index=False, header=False)
print(f"The agency that handled the highest number of service requests and its count: {first_row}")
print(f"The agency that handled the lowest number of service requests and its count: {last_row}")
print("Agency division and the count of requests handles by each division is as given below:")
display(sorted_counts.head(6))

# Group by 'agency_division' and calculate the count of requests and average response time
print("\n\033[1m"+"Agency division, Count of requests and its efficiency:"+"\033[0m")
groupedEfficiencyDF = efficiencyDF.groupby('agency_division').agg(
    request_count=('service_request_id', 'size'),
    average_response_time=('response_time', 'mean')
).reset_index()
groupedEfficiencyDF['average_response_time'] = groupedEfficiencyDF['average_response_time'].round(2)
groupedEfficiencyDF = groupedEfficiencyDF.sort_values(by='average_response_time', ascending=True)
first_row = groupedEfficiencyDF.head(1).to_string(index=False, header=False)
last_row = groupedEfficiencyDF.tail(1).to_string(index=False, header=False)
print(f"The most efficient agency and its response time: {first_row}")
print(f"The least efficient agency and its response time: {last_row}")
print("Agency division, Count of requests and its efficiency in days:")
display(groupedEfficiencyDF)


# How does the response efficiency vary across different years
print("\n\033[1m"+"Response efficiency of Agency division over years:"+"\033[0m")
average_response_time_per_year = efficiencyDF.groupby('request_year').agg(
    request_count=('service_request_id', 'size'),
    average_response_time=('response_time', 'mean')
).reset_index()
average_response_time_per_year['average_response_time'] = average_response_time_per_year['average_response_time'].round(2)
average_response_time_per_year = average_response_time_per_year.sort_values(by='request_year', ascending=True)
print("Average response efficiency per year for all agency divisions:")
display(average_response_time_per_year)


# How the response efficiency vary across different years for each agency divisions
average_response_time_per_year_and_agency = efficiencyDF.groupby(['request_year', 'agency_division']).agg(
    request_count=('service_request_id', 'size'),
    average_response_time=('response_time', 'mean')
).reset_index()
average_response_time_per_year_and_agency['average_response_time'] = average_response_time_per_year_and_agency['average_response_time'].round(2)
average_response_time_per_year_and_agency = average_response_time_per_year_and_agency.sort_values(by=['request_year', 'agency_division'], ascending=True)
#display(average_response_time_per_year_and_agency)


pivoted_df = average_response_time_per_year_and_agency.pivot(index='agency_division', columns='request_year', values=['request_count', 'average_response_time'])
display(pivoted_df)



Agency division and the count of requests handles by each division:


KeyError: 'duplicate_request'